# GROQ TRIAL

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = "Enter your own!!! ",
    model_name = "llama-3.1-70b-versatile"
    # other params...
)

response = llm.invoke("The first person to land on the moon was ...")
print(response.content)

Neil Armstrong.  He stepped onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


# CHROMA DB TRIAL

In [3]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name = "my_collection")

In [4]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids = ["id1", "id2"]
)

In [5]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['This document is about New York',
  'This document is about Delhi'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [6]:
documents = collection.get(ids=["id1"])
documents

{'ids': ['id1'],
 'embeddings': None,
 'metadatas': [None],
 'documents': ['This document is about New York'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [7]:
results = collection.query(
    query_texts = ["Query is about Chhole Bhature"],
    n_results = 2
)
results

{'ids': [['id2', 'id1']],
 'distances': [[1.5588479042053223, 1.8114912509918213]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'This document is about New York']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [8]:
results = collection.query(
    query_texts = ["Query is about Brooklyn Bridge"],
    n_results = 2
)
results

{'ids': [['id1', 'id2']],
 'distances': [[1.095342755317688, 1.571172833442688]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['This document is about New York',
   'This document is about Delhi']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [9]:
collection.delete(ids=all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [10]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids = ["id1", "id2"],
    metadatas=[
        {"url": "https://en.wikipedia.org/wiki/New_York_City"},
        {"url": "https://en.wikipedia.org/wiki/Delhi"}
    ]
)

In [11]:
results = collection.query(
    query_texts = ["Query is about Chhole Bhature"],
    n_results = 2
)
results

{'ids': [['id2', 'id1']],
 'distances': [[1.5588479042053223, 1.8114912509918213]],
 'metadatas': [[{'url': 'https://en.wikipedia.org/wiki/Delhi'},
   {'url': 'https://en.wikipedia.org/wiki/New_York_City'}]],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'This document is about New York']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

# EMAIL GEN EXPERIMENT

In [14]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
loader = WebBaseLoader("https://jobs.nike.com/de/job/R-38898?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Bewirb dich für Director of AI/Machine Learning Engineering

JobsucheNavigation überspringenJobsucheNIKE, INC. JOBSBefristete JobsJoin The Talent CommunityUnser Leben bei NikeÜbersichtLeistungenMarkenÜbersichtJordanConverseTeamsÜbersichtAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public Affairs
Human ResourcesData & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyStandorteÜbersichtNike WHQNike New York HQEHQ: Hilversum, NiederlandeELC: Laakdal, BelgiumGreater China HQDiversität, Gleichstellung und InklusionÜbersichtMilitärBehinderung und InklusionInklusion der indigenen BevölkerungPraktikaData & AnalyticsDirector of AI/Machine Learning

In [17]:
from langchain_core.prompts import PromptTemplate

In [20]:
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the careers page of a website.
    Extract the job postings and return them in JSON format containing the following keys:
    'role', 'experience', 'skills' and 'description'.
    Only return the valid JOSN.
    ### Valid JSON (NO PREAMBLE):
    """
)

In [21]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
{
  "role": "Director of AI/Machine Learning Engineering",
  "experience": "Proven ability to develop production-grade software across areas like software engineering, data engineering, or machine learning. Demonstrated success in a leadership or management role, guiding teams to achieve results.",
  "skills": [
    "AI/ML technologies such as TensorFlow, PyTorch, Scikit-learn",
    "Platforms like AWS Sagemaker and Databricks MLflow",
    "Generative AI and Foundation models (e.g., AWS Bedrock, AWS Jumpstart, Databricks Mosaic, Anthropic, OpenAI, Cohere, etc.)",
    "Deploying cloud solutions with a focus on scalability, performance, and reliability (AWS preferred)",
    "Agile environments to deliver high-quality software solutions"
  ],
  "description": "Lead the development and delivery of our GenAI Center of Excellence, GSL software, SwooshChat, and GenAI use cases. Build and grow highly skilled, cross-functional engineering teams that deliver solutions unlocking AI/ML for

In [23]:
type(res.content)

str

In [24]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Director of AI/Machine Learning Engineering',
 'experience': 'Proven ability to develop production-grade software across areas like software engineering, data engineering, or machine learning. Demonstrated success in a leadership or management role, guiding teams to achieve results.',
 'skills': ['AI/ML technologies such as TensorFlow, PyTorch, Scikit-learn',
  'Platforms like AWS Sagemaker and Databricks MLflow',
  'Generative AI and Foundation models (e.g., AWS Bedrock, AWS Jumpstart, Databricks Mosaic, Anthropic, OpenAI, Cohere, etc.)',
  'Deploying cloud solutions with a focus on scalability, performance, and reliability (AWS preferred)',
  'Agile environments to deliver high-quality software solutions'],
 'description': 'Lead the development and delivery of our GenAI Center of Excellence, GSL software, SwooshChat, and GenAI use cases. Build and grow highly skilled, cross-functional engineering teams that deliver solutions unlocking AI/ML for Nike. Develop and execute str

In [25]:
type(json_res)

dict

In [27]:
import pandas as pd

In [28]:
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [31]:
import chromadb

client = chromadb.PersistentClient('vectorstore') # Creates DB on disk rather than memory(HD).
collection = client.get_or_create_collection(name = "portfolio")

In [35]:
import uuid

In [36]:
if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas = {"links" : row["Links"] },
                       ids = [str(uuid.uuid4())] 
                    )

In [38]:
collection

Collection(id=07ba2434-b3e8-46ef-8b10-73f39d05ae1c, name=portfolio)

In [39]:
links = collection.query(query_texts = ["Experience in Python", "Expertise in React Native"], n_results = 2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [40]:
job = json_res
job['skills']

['AI/ML technologies such as TensorFlow, PyTorch, Scikit-learn',
 'Platforms like AWS Sagemaker and Databricks MLflow',
 'Generative AI and Foundation models (e.g., AWS Bedrock, AWS Jumpstart, Databricks Mosaic, Anthropic, OpenAI, Cohere, etc.)',
 'Deploying cloud solutions with a focus on scalability, performance, and reliability (AWS preferred)',
 'Agile environments to deliver high-quality software solutions']

In [41]:
links = collection.query(query_texts = job['skills'], n_results = 2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [42]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

In [43]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking AI/ML Potential for Nike with AtliQ's Expertise

Dear Hiring Manager,

I came across the job description for the Director of AI/Machine Learning Engineering role at Nike, and I was impressed by the company's vision to develop and deliver innovative AI/ML solutions. As a Business Development Executive at AtliQ, I believe our company's expertise in AI & Software Consulting can help Nike achieve its goals.

AtliQ has a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing production-grade software across areas like software engineering, data engineering, and machine learning.

We have hands-on experience with AI/ML technologies such as TensorFlow, PyTorch, and Scikit-learn, as well as platforms like AWS Sagemaker and Databricks MLflow. Our expertise in deploying cloud solutions with a focus on scalabi